<a id = Section2></a>

---
## Data Pre-Processing
---

### **1.1 Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import os
from pandas_profiling import ProfileReport

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', None)

### **1.2 Import employee_data, employee_details_data and department_data and validate**

In [2]:
path = os.getcwd()

employee = pd.read_csv(path + '\\employee_data.csv', index_col='employee_id')
department = pd.read_csv(path + '\\department_data.csv', index_col='dept_id')
employee_details= pd.read_csv(path + '\\employee_details_data.csv', index_col='employee_id')

**Check shape of tables**

In [3]:
print('department_data:', department.shape)
print('employee_data:', employee.shape)
print('employee_details_data:', employee_details.shape)

department_data: (11, 2)
employee_data: (14150, 10)
employee_details_data: (14245, 3)


### **1.3 Merge employee_data, employee_details_data and department_data in single file**

#### 1.3.1 Check data quality for index cols in department

In [4]:
employee['department'].value_counts()

D00-SS     3905
D00-ENG    2575
D00-SP     2113
D00-IT     1157
D00-PD      855
D00-MT      815
D00-FN      725
D00-MN      593
-IT         207
D00-AD      175
D00-PR      173
D00-TP      150
Name: department, dtype: int64

**Observation**
- We can see the '-IT' is incorrect and needs to be replaced with 'D00-IT'

In [5]:
employee['department'].replace(to_replace='-IT', value='D00-IT', inplace=True )

**After correcting employee['department'] values**

In [6]:
employee['department'].value_counts()

D00-SS     3905
D00-ENG    2575
D00-SP     2113
D00-IT     1364
D00-PD      855
D00-MT      815
D00-FN      725
D00-MN      593
D00-AD      175
D00-PR      173
D00-TP      150
Name: department, dtype: int64

**Merge employee_data and employee_det with employee_id as index value** 

In [7]:
data = employee.join(employee_details).reset_index()

**Rename department to dept_id to sync with col name in department and merge with department**

In [8]:
data = data.rename(columns={'department':'dept_id'}).set_index('dept_id')
data = data.join(department).reset_index()

**Validate merge**

In [9]:
data.shape

(14150, 16)

In [10]:
data.head()

,dept_id,employee_id,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,age,gender,marital_status,dept_name,dept_head
0,D00-AD,100360,274.0,NaN,0.810377,4,NaN,high,0.872978,Employed,3.0,41.0,Female,Married,Admin,Evelyn Tolson
1,D00-AD,100852,260.0,NaN,0.955429,4,NaN,low,0.615781,Employed,4.0,46.0,Male,Married,Admin,Evelyn Tolson
2,D00-AD,100876,254.0,1.0,0.971611,7,NaN,medium,0.155280,Left,4.0,45.0,Male,Married,Admin,Evelyn Tolson
3,D00-AD,100909,151.0,NaN,0.974578,5,NaN,medium,0.873819,Employed,3.0,46.0,Female,Married,Admin,Evelyn Tolson
4,D00-AD,101075,236.0,NaN,0.451697,3,NaN,medium,0.370081,Employed,2.0,24.0,Male,Unmarried,Admin,Evelyn Tolson


#### 1.3.2 Create copy of data

In [11]:
data_backup = data.copy()

In [12]:
data_backup.tail()

,dept_id,employee_id,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,age,gender,marital_status,dept_name,dept_head
14145,NaN,145459,154.0,NaN,0.938456,5,NaN,low,0.765289,Employed,4.0,42.0,Male,Married,NaN,NaN
14146,NaN,146223,181.0,NaN,NaN,3,NaN,low,0.843747,Employed,3.0,28.0,Female,Unmarried,NaN,NaN
14147,NaN,147123,242.0,1.0,0.775716,4,NaN,medium,0.870238,Employed,3.0,42.0,Male,Married,NaN,NaN
14148,NaN,147235,155.0,NaN,0.643488,4,NaN,high,0.811090,Employed,4.0,33.0,Male,Married,NaN,NaN
14149,NaN,148877,136.0,NaN,0.692963,3,NaN,high,0.792814,Employed,2.0,25.0,Male,Unmarried,NaN,NaN


### **2.1 Identification & Handling of Missing Data**

**Before Handling Null Data**

In [13]:
null_frame = pd.DataFrame(index = data.columns.values)
null_frame['Null Frequency'] = data.isnull().sum().values
percent = data.isnull().sum().values/data.shape[0]
null_frame['Missing %age'] = np.round(percent, decimals = 4) * 100
null_frame[null_frame['Null Frequency']>0].transpose()

,dept_id,filed_complaint,last_evaluation,recently_promoted,satisfaction,tenure,age,gender,marital_status,dept_name,dept_head
Null Frequency,707.0,12104.00,1487.00,13853.0,150.00,150.00,5.00,5.00,5.00,707.0,707.0
Missing %age,5.0,85.54,10.51,97.9,1.06,1.06,0.04,0.04,0.04,5.0,5.0


**Observation:**

- **dept_id, dept_name and dept_head**: Drop dept_id and dept_head as they giving same information; Change NaN to Others &rarr; Action Required later
- **filed_complaint**: Missing Information (12080) &rarr; Replace with- 0.
- **recently_promoted**: Missing Informa tion (13824) &rarr; Replace with- 0.
- **last_evaluation**: Missing Information (1487) &rarr; Replace with- Median value.
- **satisfaction**: Missing Information (150) &rarr; Replace with- Median value.
- **age**: Missing Informa tion (5) &rarr; Replace with- Median value.
- **gender**: Missing Information (5) &rarr; Replace with- Mode.
- **marital_status**: Missing Informa tion (5) &rarr; Replace with- Mode.
- **tenure**: Missing Information (150) &rarr; Action Required.

**Performing Operations**

In [14]:
data.drop(columns=['dept_id','dept_head','employee_id'], inplace=True)

In [15]:
# replace filed_comlaint and recently_promoted with '0'
data['filed_complaint'] = data['filed_complaint'].fillna(0)
data['recently_promoted'] = data['recently_promoted'].fillna(0)

# replace dept_name with 'Other'
data['dept_name'] = data['dept_name'].fillna('Other')

# replace age, last_evaluation, satisfaction with their 'Mean'
data['age'] = data['age'].fillna(data['age'].mean())
data['last_evaluation'] = data['last_evaluation'].fillna(data['last_evaluation'].mean())
data['satisfaction'] = data['satisfaction'].fillna(data['satisfaction'].mean())

# replace gender, tenure and marital_status with 'Mode'
data['gender'] = data['gender'].fillna(data['gender'].mode()[0])
data['tenure'] = data['tenure'].fillna(data['tenure'].mode()[0])
data['marital_status'] = data['marital_status'].fillna(data['marital_status'].mode()[0])

In [16]:
null_frame = pd.DataFrame(index = data.columns.values)
null_frame['Null Frequency'] = data.isnull().sum().values
percent = data.isnull().sum().values/data.shape[0]
null_frame['Missing %age'] = np.round(percent, decimals = 4) * 100
null_frame[null_frame['Null Frequency']>0].transpose()

""
Null Frequency
Missing %age


### **2.2 Identification & Handling of Redundant Data**
- In this section **we will identify redundant rows and columns** in our data if present.

#### **2.2.1 Identfication & Handling of Redundant Rows**

**Before Handling Duplicate Rows**

In [17]:
print('Contains Redundant Records?:', data.duplicated().any())
print('Duplicate Count:', data.duplicated().sum())

Contains Redundant Records?: True
Duplicate Count: 29


**Observation:**

- We can see that **there are redundant records present** in the data

**Performing Operations**

In [18]:
before_shape = data.shape
print('Data Shape [Before]:', data.shape)

data.drop_duplicates(inplace = True)

after_shape = data.shape
print('Data Shape [After]:', after_shape)

drop_nums = before_shape[0] - after_shape[0]
drop_percent = np.round(drop_nums / before_shape[0], decimals = 6) * 100

print('Drop Ratio:', drop_percent, '%')

Data Shape [Before]: (14150, 13)
Data Shape [After]: (14121, 13)
Drop Ratio: 0.2049 %


**After Handling Duplicate Rows**

In [19]:
print('Contains Redundant Records?:', data.duplicated().any())
print('Duplicate Count:', data.duplicated().sum())

Contains Redundant Records?: False
Duplicate Count: 0


### 3.1 Save clean data as csv file

In [20]:
path = os.getcwd()
data.to_csv(path+'\\pre_processing.csv', index=False)